In [1]:
plt.close('all')

NameError: name 'plt' is not defined

In [2]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
def stock_data(startd, endd, tickers=tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def ehlers_simple_decycle(df, src, hpLength, t):
    #hp = highpass
    upper = 0.5
    lower = 0.5

    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    df[('Alpha', t)] = 0        
    df[('Hp', t)] = 0   
    df[('Decycler', t)] = 0  
    for i in range(125, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, ('Alpha', t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, ('Alpha', t)] = df.loc[i-1, ('Alpha', t)]
        #Decycling::: src = the input!
        df.loc[i, ('Hp', t)] = np.square(1-(df.loc[i, ('Alpha', t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, ('Alpha', t)]) * df.loc[i-1, ('Hp', t)] - np.square(1-df.loc[i, ('Alpha', t)]) * df.loc[i-2, ('Hp', t)]
        df.loc[i, ('Decycler', t)] = src[i] - df.loc[i, ('Hp', t)]
    return df
    
def bolinger_bands(df, close, length, multiple):
    for i in range(len(df)):
        mean_bb = close.rolling(length).mean()
        std  = close.rolling(length).std()
        upper_bb = mean_bb + (std*multiple)
        lower_bb = mean_bb - (std*multiple)
        return mean_bb, upper_bb, lower_bb   
    
#Functions Average True Range#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, t , n, multiplier):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
#De Stoploss 2xATR implementatie:
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * multiplier)
    return Stoploss 
#End Function#

###########Legacy code::
def smoothing_sma_savgol(SMA, t):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        savgol_len = len(dfsma[(SMA + '_change', t)].dropna())
        if (savgol_len % 2) == 0:
            savgol_len += 1
            if (savgol_len % 2) != 0:
                return savgol_len
            else:
                continue
    #No real rules for polyorder, just use what fits best for its purpose!
        savgol = savgol_filter(dfsma[(SMA + '_change', t)].dropna(), savgol_len, polyorder=100)
        insert_len =  len(dfsma[(SMA + '_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        savgol = pd.Series(savgol, index=dfsma.index)
        dfsma[(SMA + '_change_smooth_savgol', t)] = savgol
    
def polyfit_df(df, t):
    dftemp = df.copy()
    for t in tickers:
        result = np.polyfit(dftemp[('Percent_return', t)].dropna().index, dftemp[('Percent_return', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
    dftemp = dftemp.reset_index()
    return dftemp

def smoothing_sma_sma(SMA, t):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_sma', t)] = dfsma[(SMA + '_change', t)].rolling(x).mean()
        
def smoothing_sma_ema(SMA, t):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_ema', t)] = dfsma[(SMA + '_change', t)].ewm(span=x, adjust=False).mean()
##############

In [17]:
################                    ALLE INPUTS (Dataframes)

start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
df2 = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
print(df.index.max())

###Needed for SELLING
#BB with 2 std 20ma
for t in tickers:
    df[('Mean_bb', t)], df[('Upper_bb', t)], df[('Lower_bb', t)] = bolinger_bands(df, df[('Close', t)], 100, 2.5) 

#ATR #1  
for t in tickers:
    df[('Stoploss_atr2', t)] = atr(df, t, 10, 2)
#ATR #2  
for t in tickers:
    df[('Stoploss_atr1', t)] = atr(df, t, 10, 1)

#Decylcer   
for t in tickers:
    df[('Percent_change_100', t)] = df[('Percent_return', t)].rolling(100).mean()
for t in tickers:
    df = ehlers_simple_decycle(df, df[('Percent_change_100', t)], 25, t)

    
    
###Needed for BUYING
#Decylcer
for t in tickers:
    df2 = ehlers_simple_decycle(df2, df2[('Close', t)], 75, t)
    
#Volume
for t in tickers:
    df2[('Volume_25', t)] = df2[('Volume', t)].rolling(25).mean()
    for i in range(len(df2)):
        if df2.loc[i, ('Volume', t)] >= df2.loc[i, ('Volume_25', t)]*1.5:
            df2.loc[i, ('Volume_check', t)] = 1
        else:
            df2.loc[i, ('Volume_check', t)] = 0

#Sma10
for t in tickers:
    df2[('SMA10', t)] = df2[('Close', t)].rolling(10).mean()
    for i in range(len(df)):
        try:
            if (df2.loc[i, ('SMA10', t)]-df2.loc[i-1, ('SMA10', t)])/df2.loc[i-1, ('SMA10', t)] > 0:
                df2.loc[i, ('SMA10_signal', t)] = 1
            else:
                df2.loc[i, ('SMA10_signal', t)] = 0
        except:
            pass
    
time = datetime.datetime.now().time()
print(time)

#for t in tickers:
    #fig, ax = plt.subplots(1, sharex=True, figsize=(10,7))#, gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.02})
    #ax[0].set_title(t)
#Decycler Sell
    #ax[0].plot(df[('Decycler', t)])
    #ax[1].plot(df[('Decycler_signal', t)])
#BB Sell
    #ax.plot(df[('Upper_bb', t)])
    #ax.plot(df[('Mean_bb', t)])
    #ax.plot(df[('Lower_bb', t)])
    #ax.plot(df[('Close', t)])

00:08:21.911926
1005
00:08:32.235722


In [18]:
############################              ALLE SIGNALS!!!!
#maak eerst alles even los van elkaar!

df[('Strat_atr_s', t)] = 0
df[('Strat_bb_s', t)] = 0

for t in tickers:
    #ATR Sell #1
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss2', t)] = df.loc[x:i, ('Stoploss_atr2', t)].max()
        if df.loc[i, ('Stoploss2', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i+1:i+5, ('Strat_atr2_s', t)] = 0
            next(islice(loop, 4, 4), None)
            x=i
        if df.loc[i, ('Stoploss2', t)] < df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat_atr2_s', t)] = 1
        else:
            df.loc[i, ('Strat_atr2_s', t)] = 1
    #ATR Sell #2
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat_atr1_s', t)] = 0
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat_atr1_s', t)] = 1
        else:
            df.loc[i, ('Strat_atr1_s', t)] = 1
    #BB Sell
    loop = iter(range(len(df)))
    for i in loop:
        if df.loc[i, ('Upper_bb', t)] < df.loc[i, ('Close', t)]:
            #df.loc[i+1:i+10, ('Strat_bb_s', t)] = 0      <-- each break out counts for ten days, better way fo doing this
            #next(islice(loop, 9, 9), None)
            df.loc[i+1, ('Strat_bb_s', t)] = 0
        if df.loc[i, ('Upper_bb', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat_bb_s', t)] = 1
    #Decycler Sell
    #Decycler moet 1 stapje skippen in range sinds ik "i-1" moet gebruiken
    for i in range(1, len(df)):
        if df.loc[i, ('Decycler', t)] >= df.loc[i-1, ('Decycler', t)]:  
            #buy/hold signaal
            df.loc[i, ('Strat_decycle_signal_s', t)] = 1
        else:
            df.loc[i, ('Strat_decycle_signal_s', t)] = 0
    
    #Decycle Buy
    for i in range(1, len(df)-1):
        if df2.loc[i, ('Decycler', t)] >= df2.loc[i-1, ('Decycler', t)]: #and df2.loc[i-1, ('Decycler', t)] >= df2.loc[i-2, ('Decycler', t)]:
            df2.loc[i, ('Strat_decycle_signal_b', t)] = 1
        else:
            df2.loc[i, ('Strat_decycle_signal_b', t)] = 0
    #Volume Buy  <----No Longer in Use
    
    
    #for i in range(len(df2)):
     #   if df2.loc[i, ('Skew_log_volume', t)] >= 1.04:
      #      df2.loc[i, ('Strat_log_volume_signal', t)] = 1
       # else:
        #    df2.loc[i, ('Strat_log_volume_signal', t)] = 0
    #SMA10 Buy
    #Percent change moet 1 stapje skippen in range sinds ik "i-1" moet gebruiken
    df2[('SMA10', t)] = df2[('Close', t)].rolling(10).mean()
    for i in range(1, len(df)-1):
        if (df2.loc[i, ('SMA10', t)]-df2.loc[i-1, ('SMA10', t)])/df2.loc[i-1, ('SMA10', t)] > 0:
            df2.loc[i, ('Strat_SMA10_signal', t)] = 1
        else:
            df2.loc[i, ('Strat_SMA10_signal', t)] = 0

            
#for t in tickers:
#    fig, ax = plt.subplots(1, sharex=True, figsize=(8,5))
#    ax.plot(df2[('Skew_log_volume', t)])
 

In [19]:
#Sell Strat
for t in tickers:
    df.loc[:100, ('Strat_bb_s', t)]=1 #Omdat BB een 100MA gebruikt!
    df[('Strat_s', t)] = 0
    for i in range(len(df)-1):
        try:
      #      #BB expirimenteren
       #     if df.loc[i, ('Strat_bb_s', t)]==0 or df.loc[i-1, ('Strat_bb_s', t)]==0 or df.loc[i-2, ('Strat_bb_s', t)]==0 or df.loc[i-3, ('Strat_bb_s', t)]==0 or df.loc[i-4, ('Strat_bb_s', t)]==0 or df.loc[i-5, ('Strat_bb_s', t)]==0 or df.loc[i-5, ('Strat_bb_s', t)]==0 or df.loc[i-6, ('Strat_bb_s', t)]==0 or df.loc[i-7, ('Strat_bb_s', t)]==0 or df.loc[i-7, ('Strat_bb_s', t)]==0 or df.loc[i-8, ('Strat_bb_s', t)]==0 or df.loc[i-9, ('Strat_bb_s', t)]==0 or df.loc[i-10, ('Strat_bb_s', t)]==0:
        #        if df.loc[i, ('Strat_atr1_s', t)] == 0 or df.loc[i-1, ('Strat_atr1_s', t)] == 0 or df.loc[i-2 ('Strat_atr1_s', t)] == 0 or df.loc[i-3 ('Strat_atr1_s', t)] == 0 or df.loc[i-4 ('Strat_atr1_s', t)] == 0 or df.loc[i-5, ('Strat_atr1_s', t)] == 0:
         #           df.loc[i, ('Strat_s', t)] = 1
                    
            if df.loc[i, ('Strat_decycle_signal_s', t)] == 0:
                if df.loc[i, ('Strat_atr2_s', t)] == 0 or df.loc[i-1, ('Strat_atr2_s', t)] == 0 or df.loc[i-2 ('Strat_atr2_s', t)] == 0 or df.loc[i-3 ('Strat_atr2_s', t)] == 0 or df.loc[i-4 ('Strat_atr2_s', t)] == 0 or df.loc[i-5, ('Strat_atr2_s', t)] == 0:
                    df.loc[i, ('Strat_s', t)] = 1
        except:
            pass
#Buy strat  
for t in tickers:
    df2[('Strat_b', t)] = 0
    for i in range(len(df)-1):
        if df2.loc[i, ('Strat_decycle_signal_b', t)] == 1:
                df2.loc[i, ('Strat_b', t)] = 1   
                
#Bringing it together
for t in tickers:
    i=0
    df[('Strat', t)] = 0
    while i < len(df)-1:
        try: 
            if df.loc[i, ('Strat_s', t)] == 1:
                while True:
                    if df2.loc[i, ('Strat_b', t)] == 0:
                        df.loc[i+1, ('Strat', t)] = 0
                        i+=1
                    if df2.loc[i, ('Strat_b', t)] == 1:
                        df.loc[i+1, ('Strat', t)] = 1
                        i+=1
                        break               
            if df.loc[i, ('Strat_s', t)] == 0:
                df.loc[i+1, ('Strat', t)] = 1
                i+=1
                
        ##Add an x day required NO-SELL period; to prevent sawtooths
        except:
            pass

df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
    
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()

for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5, 0.5, 0.5], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)])
    ax[1].plot(df[('Strat', t)])
    ax[2].plot(df[('Percent_return', 'Total')].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    ax[3].plot(df2[('Strat_b', t)])
    ax[4].plot(df[('Strat_s', t)])
    
r1=df[('Strat_return', tickers[0])].sum()
r2=df[('Percent_return', tickers[0])].sum()
print(f'Strat return SPY = {r1}')
print(f'Hold return SPY = {r2}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.48723185914568157
Hold return SPY = 0.5041826175961925


In [6]:
print(df.index.max())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df) 
#display(df2)

1006


Attributes       Date   Adj Close       Close        High         Low  \
Symbols                       SPY         SPY         SPY         SPY   
0          2016-01-04  182.513626  201.020004  201.029999  198.589996   
1          2016-01-05  182.822327  201.360001  201.899994  200.050003   
2          2016-01-06  180.516174  198.820007  200.059998  197.600006   
3          2016-01-07  176.185333  194.050003  197.440002  193.589996   
4          2016-01-08  174.251419  191.919998  195.850006  191.580002   
5          2016-01-11  174.423950  192.110001  193.410004  189.820007   
6          2016-01-12  175.831238  193.660004  194.550003  191.139999   
7          2016-01-13  171.445892  188.830002  194.860001  188.380005   
8          2016-01-14  174.260468  191.929993  193.259995  187.660004   
9          2016-01-15  170.519791  187.809998  188.759995  185.520004   
10         2016-01-19  170.746765  188.059998  190.110001  186.199997   
11         2016-01-20  168.558640  185.649994  187.500000  181.020004   
12         2016-01-21  169.502914  186.690002  188.869995  184.639999   
13         2016-01-22  172.980286  190.520004  190.759995  188.880005   
14         2016-01-25  170.365448  187.639999  190.149994  187.410004   
15         2016-01-26  172.689743  190.199997  190.529999  188.020004   
16         2016-01-27  170.810318  188.130005  191.559998  187.059998   
17         2016-01-28  171.700104  189.110001  190.199997  187.160004   
18         2016-01-29  175.885727  193.720001  193.880005  189.880005   
19         2016-02-01  175.822144  193.649994  194.580002  191.839996   
20         2016-02-02  172.653442  190.160004  191.970001  189.539993   
21         2016-02-03  173.688477  191.300003  191.779999  187.100006   
22         2016-02-04  173.960876  191.600006  192.750000  189.960007   
23         2016-02-05  170.646927  187.949997  191.669998  187.199997   
24         2016-02-08  168.349808  185.419998  186.119995  182.800003   
25         2016-02-09  168.358871  185.429993  186.940002  183.199997   
26         2016-02-10  168.213638  185.270004  188.339996  185.119995   
27         2016-02-11  166.025482  182.860001  184.100006  181.089996   
28         2016-02-12  169.448425  186.630005  186.649994  183.960007   
29         2016-02-16  172.308426  189.779999  189.809998  187.630005   
30         2016-02-17  175.123047  192.880005  193.320007  191.009995   
31         2016-02-18  174.405762  192.089996  193.270004  191.720001   
32         2016-02-19  174.324066  192.000000  192.179993  190.449997   
33         2016-02-22  176.848099  194.779999  194.949997  193.789993   
34         2016-02-23  174.614578  192.320007  194.320007  192.179993   
35         2016-02-24  175.413559  193.199997  193.529999  189.320007   
36         2016-02-25  177.538147  195.539993  195.550003  192.830002   
37         2016-02-26  177.129562  195.089996  196.679993  194.899994   
38         2016-02-29  175.740417  193.559998  196.229996  193.330002   
39         2016-03-01  179.871582  198.110001  198.210007  194.449997   
40         2016-03-02  180.679596  199.000000  199.059998  197.250000   
41         2016-03-03  181.387802  199.779999  199.800003  198.110001   
42         2016-03-04  181.977966  200.429993  201.350006  199.029999   
43         2016-03-07  182.123245  200.589996  201.070007  199.250000   
44         2016-03-08  180.134842  198.399994  199.919998  198.210007   
45         2016-03-09  181.024643  199.380005  199.789993  198.429993   
46         2016-03-10  181.169907  199.539993  201.070007  197.380005   
47         2016-03-11  184.093460  202.759995  202.809998  199.520004   
48         2016-03-14  183.857391  202.500000  203.039993  201.770004   
49         2016-03-15  183.557770  202.169998  202.529999  201.050003   
50         2016-03-16  184.620056  203.339996  203.820007  201.550003   
51         2016-03-17  185.791290  204.630005  205.229996  202.770004   
52         2016-03-18  186.521408  204.380005  204.779999 

In [ ]:
start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
print(df.index.max())
### If Close is 2xATR lager, SELL 3 dagen lang;   Meest basic implementatie van stoploss! 


for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    
#   Do-untill code:  https://stackoverflow.com/questions/1662161/is-there-a-do-until-in-python

df[('Stoploss', 'SPY')] = 0
x=0
for t in tickers:
    loop = iter(range(len(df)))
    df[('ATR', t)] = atr(df, t, 1, 2)
    for i in loop:
        df.loc[i, ('Stoploss', t)] = df.loc[x:i, ('ATR', t)].max()
        if df.loc[i, ('Stoploss', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i+1:i+3, ('Strat', t)] = 0
            x=i+3
            next(islice(loop, 2, 2), None)
        if df.loc[i, ('Stoploss', t)] < df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat', t)] = 1

###

df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
df = df.dropna()
df = df.reset_index()
for t in tickers:
    fig, ax = plt.subplots(3, sharex=True, figsize=(10,7), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5], 'hspace':0.02})
    ax[0].set_title(t)
    #ax[0].plot(df[('Percent_return', t)], linewidth=0.5)
    ax[0].plot(df[('Close', t)], linewidth=1.5)
    ax[0].plot(df[('Stoploss', t)], linewidth=1.5)
    ax[1].plot(df[('Strat', t)], linewidth=0.5)
    ax[2].plot(df[('Percent_return', 'Total')].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    #ax[0].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
r1=df[('Strat_return', tickers[0])].sum()
r2=df[('Percent_return', tickers[0])].sum()
print(f'Strat return SPY = {r1}')
print(f'Hold return SPY = {r2}')
end_time = datetime.datetime.now().time()
print(f'Time when finished:                   {end_time}\n')
print(df.index.max())

In [181]:
for t in tickers:
    df2[('Log_volume', t)] = np.log(df2[('Volume', t)]) - np.log(df2[('Volume', t)].shift(1))
    for i in range(len(df)-1):
        dist = np.asarray(df2.loc[i-20:i, ('Log_volume', t)])
        df2.loc[i, ('Log_volume_skew', t)] = stats.skew(dist)
    
df2[('cv', 'SPY')] = 1.004
display(df2[('Log_volume_skew', 'SPY')])
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(8,5))
    ax[0].plot(df2[('Close', t)])
    ax[1].plot(df2[('Log_volume_skew', t)])
    ax[1].plot(df2[('cv', t)])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1001   -0.325659
1002   -0.364168
1003   -0.511747
1004   -0.509184
1005   -0.611675
Name: (Log_volume_skew, SPY), Length: 1006, dtype: float64

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …